In [ ]:
# -*- coding:utf-8 -*-

import math
import numpy as np
import cv2
from matplotlib import pyplot as plt

# 写一个且分图片的程序，面向的这个图片是 cubic 立体的，输出也是 cubic
# end_row, end_col 是不可以取得到的
def GetSubCubic(pic_cubic, (start_row, end_row, start_col, end_col)):
    num_rows = end_row - start_row
    num_cols = end_col - start_col
    layer = pic_cubic.shape

    if 2 == len(layer):  # normal mxn picture
        sub_Cubic = np.zeros((num_rows, num_cols), dtype=np.uint16)
        sub_Cubic[:,:] = pic_cubic[start_row:end_row, start_col:end_col]

    else:
        sub_Cubic = np.zeros((num_rows, num_cols, layer[2]), dtype=np.uint16)
        sub_Cubic[:, :, :] = pic_cubic[start_row:end_row, start_col:end_col,:]

    return sub_Cubic

# This is For Fourier Transfarm
def Fouries(pic_cubic):
    f = np.fft.fft2(pic_cubic)
    fshift = np.fft.fftshift(f)
    magnitude_spectrum = 20 * np.log(np.abs(fshift))
    return magnitude_spectrum


# 这个函数将矩阵拉长成向量
# One: Square(nxn) Pic To (nxn)x1 Vector
def SquareToVector(square_pic):
    m, n = square_pic.shape
    if m != n:
        print('In SquareToVector: m is not equal to n, Error')

    vec = np.zeros((m*n,1))

    #fill the vec
    for row in range(0,m):
        for col in range(0,n):
            vec[row*m+col] = square_pic[row,col]

    return vec


# 这个函数将向量变成矩阵
# Two: (nxn)x1 Vector To Square(nxn)
def VectorToSquare(vec):
    n = len(vec)
    m = int(math.sqrt(n))
    if m*m != n:
        print('In VectorToSquare: m*m is not equal to n, Error, Not A Square')

    square = np.zeros((m,m))

    #fill the square
    for it in range(0,n):
        row = int(it/m)
        col = it%m
        square[row,col] = vec[it]

    return square


# 写一个函数，生成我的
def GenerateSamples( Old_pic, New_pic, Target_pic, inner_square, U_L_D_R ):

    # inner square
    m,n = inner_square.shape
    if m != n:
        print('Error Inner Pic Must Be Square.')
        exit(1)

    # Up Left Down Right
    Up = U_L_D_R[0]
    Left = U_L_D_R[1]
    Down = U_L_D_R[2]
    Right = U_L_D_R[3]

    # smaller picture width and height
    small_width = Left + n + Right
    small_height = Up + n + Down

    # smaller to OUT picture width and height
    out_width = 2 * small_width
    out_height = 2 * small_height

    # Shapes
    sp = Old_pic.shape
    width = sp[0]
    height = sp[1]
    depth = sp[2]

    # 水平个数 垂直个数
    num_horizion = int( ( width - Left - Right ) / n )
    num_vertical = int( ( height - Up - Down ) / n )

    for ver in range(0, num_vertical):
        for hor in range(0,num_horizion):
            start_row = ver * n
            end_row = start_row + Up + n + Down
            start_col = hor * n
            end_col = start_col + Left + n + Right

            strtend = (start_row, end_row, start_col, end_col)

            # Get From 2015 左上
            tmp_old = GetSubCubic(Old_pic, strtend)

            # 图像处理, 2015 频率域 右上
            tmp_old_fft = Fouries(tmp_old)

            # Get From 2017 右下
            tmp_new = GetSubCubic(New_pic, strtend)

            # 图像处理, 2017 频率域 左下
            tmp_new_fft = Fouries(tmp_new)

            # 合成完整一个图片
            sample_pic = np.zeros((out_width, out_height, depth))

            sample_pic[0:out_height, 0:out_width, :] = tmp_old

            sample_pic[0:out_height, out_width:, :] = tmp_old_fft

            sample_pic[out_height:, 0:out_width, :] = tmp_new_fft

            sample_pic[out_height, out_width:, :] = tmp_new

            # Get From Half Fake Target...
            srow_square = start_row + Up
            scol_square = start_col + Left
            tmp_target = GetSubCubic(Target_pic, (srow_square,srow_square+n,scol_square,scol_square+n))
            tar_vet = SquareToVector(tmp_target)

            # 保存图片样本 目标样本



# 这个函数将小矩阵合并成大矩阵，作为输出，输出就是 5106x15106

if __name__ == '__main__':
    print('hello world')

    a_mat = np.random.rand(5,5)

    v = SquareToVector(a_mat)

    print(a_mat)
    print(v)

    s = VectorToSquare(v)

    print(s)